In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import json

In [6]:
ds = xr.open_dataset('/home/bmurphy/cmaq_projects/research_20250123/data/output_CCTM_v55_intel23.2_Bench_2016_12SE1/CCTM_PHOTDIAG2_v55_intel23.2_Bench_2016_12SE1_20160701.nc')

In [7]:
ds

<xarray.Dataset> Size: 860MB
Dimensions:         (TSTEP: 24, VAR: 32, DATE-TIME: 2, LAY: 35, ROW: 80,
                     COL: 100)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables: (12/33)
    TFLAG           (TSTEP, VAR, DATE-TIME) int32 6kB ...
    NO2_IUPAC10     (TSTEP, LAY, ROW, COL) float32 27MB ...
    O3_O3P_IUPAC10  (TSTEP, LAY, ROW, COL) float32 27MB ...
    O3_O1D_IUPAC10  (TSTEP, LAY, ROW, COL) float32 27MB ...
    H2O2_IUPAC10    (TSTEP, LAY, ROW, COL) float32 27MB ...
    NO3NO2_06       (TSTEP, LAY, ROW, COL) float32 27MB ...
    ...              ...
    FMCL_IUPAC04    (TSTEP, LAY, ROW, COL) float32 27MB ...
    CLNO2_IUPAC13   (TSTEP, LAY, ROW, COL) float32 27MB ...
    CLONO2_1        (TSTEP, LAY, ROW, COL) float32 27MB ...
    CLONO2_2        (TSTEP, LAY, ROW, COL) float32 27MB ...
    IC3ONO2         (TSTEP, LAY, ROW, COL) float32 27MB ...
    ACRO_09         (TSTEP, LAY, ROW, COL) float32 27MB ...
Attributes: (12/33)
    IOAPI_VERSION:  ioapi-3.2: $Id: init3.F90 247 2023-03-22 15:59:19Z coats ...
    EXEC_ID:        CMAQ_CCTMv55_sha=d0d1a121ff_bmurphy_20250228_162353_45291...
    FTYPE:          1
    CDATE:          2025059
    CTIME:          162356
    WDATE:          2025059
    ...             ...
    VGLVLS:         [1.     0.9975 0.995  0.99   0.985  0.98   0.97   0.96   ...
    GDNAM:          2016_12SE1      
    UPNAM:          OPPHOT          
    VAR-LIST:       NO2_IUPAC10     O3_O3P_IUPAC10  O3_O1D_IUPAC10  H2O2_IUPA...
    FILEDESC:       Three dimensional values of Photolysis rates             ...
    HISTORY:

In [8]:
with open('/home/dquevedo/AMBRS/ambrs_mam4_cb6r5_ae7_aq/tests/mechanism.json','r') as f:
    data = json.load(f)
df = pd.DataFrame(dict(data)['camp-data'][0]['reactions'])

In [9]:
dat = df.notes.dropna().str.split('/').explode()

In [10]:
dat

7                   1.0
7      <O3_O3P_IUPAC10>
8                   1.0
8      <O3_O1D_IUPAC10>
19                  1.0
             ...       
215    <ALD2_R_IUPAC13>
222                 1.0
222           <ACRO_09>
227                 1.0
227           <ACRO_09>
Name: notes, Length: 84, dtype: object

In [17]:
str_dat = ''''''
for i in range(0, dat.size, 2):
    label = dat.iloc[i+1][1:-1]
    if label in str_dat:
        continue
    if float(dat.iloc[i]) == 0.0:
        j = 0
    else:
        j = float(ds.isel(COL=70, ROW=64, LAY=0, TSTEP=12)[label].data)
    str_dat += \
f'''
{label}, {float(dat.iloc[i])*j/60.0:.4e}
'''
print(str_dat)


O3_O3P_IUPAC10, 2.3785e-04

O3_O1D_IUPAC10, 8.2234e-06

H2O2_IUPAC10, 3.3740e-06

NO3NO2_06, 1.1233e-01

NO3NO_06, 1.4122e-02

N2O5_IUPAC10, 1.8780e-05

HONO_IUPAC10, 7.3398e-04

HNO3_IUPAC10, 2.8521e-07

PNA_IUPAC10, 2.4185e-06

PAN_IUPAC10, 2.9931e-07

MEPX_IUPAC10, 2.3170e-06

NTR_IUPAC10, 1.1431e-06

NITROX_BUTA_2014, 0.0000e+00

NITROX_ACET_2014, 0.0000e+00

FORM_R_IUPAC13, 1.1592e-05

FORM_M_IUPAC13, 2.2087e-05

ALD2_R_IUPAC13, 1.6660e-06

ALDX_R_IUPAC13, 2.0889e-05

GLYD_IUPAC13, 2.2219e-06

GLY_R_IUPAC13, 2.4713e-05

MGLY_IUPAC10, 6.5398e-05

KET_IUPAC10, 2.1450e-07

ACET_IUPAC10, 1.6520e-07

ISPD, 1.1584e-05

HPALD, 2.2200e-04

NO2_IUPAC10, 2.1066e-04

CL2_IUPAC04, 1.0437e-03

HOCL_IUPAC04, 1.1648e-04

FMCL_IUPAC04, 3.7031e-08

CLNO2_IUPAC13, 1.5425e-04

CLONO2_1, 3.0143e-06

CLONO2_2, 1.7980e-05

IC3ONO2, 9.7241e-07

ACRO_09, 9.7359e-07



In [18]:
with open('/home/dquevedo/AMBRS/ambrs_mam4_cb6r5_ae7_aq/tests/phot_j_labeled.dat','w') as f:
    f.write(str_dat)

In [19]:
pd.read_csv('/home/dquevedo/AMBRS/ambrs_mam4_cb6r5_ae7_aq/tests/phot_j_labeled.dat', header=None, index_col=0)

,1
0,
O3_O3P_IUPAC10,2.378500e-04
O3_O1D_IUPAC10,8.223400e-06
H2O2_IUPAC10,3.374000e-06
NO3NO2_06,1.123300e-01
NO3NO_06,1.412200e-02
N2O5_IUPAC10,1.878000e-05
HONO_IUPAC10,7.339800e-04
HNO3_IUPAC10,2.852100e-07
PNA_IUPAC10,2.418500e-06


In [7]:
None == None

True